In [17]:
from cmapPy.pandasGEXpress.parse import parse
import pandas as pd
import numpy as np

In [18]:
data = parse("C:/Users/ngomi/Downloads/Documents/data.gctx").data_df
data.index = data.index.astype(int)
gene_info = pd.read_csv("../data/geneinfo_beta.txt", delimiter="\t")
gene_dict = {}
for i in range(len(gene_info.index)):
    gene_dict[str(gene_info["gene_symbol"][i])] = gene_info["gene_id"][i]

In [19]:
up_gene_file = "../data/Example_up_genes.txt"
down_gene_file = "../data/Example_down_genes.txt"

def read_genes(file_name):
    with open(file_name) as file:
        return [gene.strip() for gene in file.readlines()]

up_genes = read_genes(up_gene_file)
down_genes = read_genes(down_gene_file)
n_expression = len(data.columns)
n_ref_gene = len(data.index)

In [20]:
def calc_es_score(ref_df, cid, gene_list):
    gene_indexes = ref_df.index.isin(gene_list)
    ns = len(gene_list)
    nr = ref_df.loc[gene_indexes, cid].sum()
    cumsum_score = np.zeros(n_ref_gene)
    cumsum_score[gene_indexes] = ref_df.loc[gene_indexes, cid] / nr
    cumsum_score[~gene_indexes] = -1 / (n_ref_gene - ns)
    scores = np.cumsum(cumsum_score)
    min_score = min(scores)
    max_score = max(scores)
    return max_score if abs(max_score) > abs(min_score) else min_score

In [21]:

import time

def cmap(up_gene_list, down_gene_list):
    up_gene_list = [gene_dict[i] for i in up_gene_list if i in gene_dict]
    down_gene_list = [gene_dict[i] for i in down_gene_list if i in gene_dict]
    c_scores = []
    start = time.time()
    for cid in data.columns:
        df = pd.DataFrame(data[cid].sort_values(ascending=False)).abs()
        up_es_score = calc_es_score(df, cid, up_gene_list)
        down_es_score = calc_es_score(df, cid, down_gene_list)
        if up_es_score*down_es_score < 0:
            c_scores.append((up_es_score - down_es_score)/2)
        else:
            c_scores.append(0)
    end = time.time()
    print("Running time", (end - start)*1000)
    return pd.DataFrame({"expression": data.columns, "c_score": c_scores})

In [22]:
result = cmap(up_genes, down_genes)
result = result.sort_values(by=["c_score"], ascending=False)
result.to_csv("test result.csv")

Running time 170871.50835990906
